In [ ]:
from dotenv import load_dotenv
# Load environmental variables
load_dotenv('../.env')

import pandas as pd
import plaid
from plaid.api import plaid_api
from plaid.model.transactions_get_request import TransactionsGetRequest
from plaid.model.transactions_get_request_options import TransactionsGetRequestOptions
from datetime import datetime
import json
import os
from dataScripts.dataAcquisition import getAccountsForUsername

In [ ]:
clientId = os.getenv('PLAID_CLIENTID')
secret = os.getenv('PLAID_SECRET')
username = os.getenv('SAMPLE_USERNAME')

In [ ]:
accountRecords = getAccountsForUsername(username)
accountRecord = accountRecords[0]
accountRecord

In [ ]:
accessToken = accountRecord['PlaidAccessCode']

In [ ]:
# Available environments are
# 'Production'
# 'Development'
# 'Sandbox'
configuration = plaid.Configuration(
    host=plaid.Environment.Development,
    api_key={
        'clientId': clientId,
        'secret': secret,
    }
)
api_client = plaid.ApiClient(configuration)
client = plaid_api.PlaidApi(api_client)

In [ ]:
request = TransactionsGetRequest(
            access_token=accessToken,
            start_date=datetime.strptime('2023-01-01', '%Y-%m-%d').date(),
            end_date=datetime.strptime('2024-01-01', '%Y-%m-%d').date(),
            options=TransactionsGetRequestOptions()
)
response = client.transactions_get(request)
transactions = response['transactions']

# # Manipulate the count and offset parameters to paginate
# # transactions and retrieve all available data
# while len(transactions) < response['total_transactions']:
#     request = TransactionsGetRequest(
#                 access_token=access_token,
#                 start_date=datetime.strptime('2020-01-01', '%Y-%m-%d').date(),
#                 end_date=datetime.strptime('2023-01-01', '%Y-%m-%d').date(),
#                 options=TransactionsGetRequestOptions(
#                 offset=len(transactions)
#             )
#     )
#     response = client.transactions_get(request)
#     transactions.extend(response['transactions'])

In [ ]:
len(transactions)

In [ ]:
transactions_dict_list = []
for tx in transactions:
    transactions_dict_list.append(tx.to_dict())

In [ ]:
# the json file where the output must be stored
transactions_json = 'debug_transactions.json'
out_file = open(transactions_json, "w")
json.dump(transactions_dict_list, out_file, default=str)
out_file.close()

In [ ]:
transactions_dict_list[0]

In [ ]:
transactions_df = pd.DataFrame.from_dict(transactions_dict_list)

In [ ]:
transactions_df.head(5)